In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical


In [3]:
# Constants
IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 100

In [4]:
def load_data(directory):
    images = []
    labels = []
    for label, gender in enumerate(['male', 'female']):
        path = os.path.join(directory, gender)
        for img_name in os.listdir(path):
            img = cv2.imread(os.path.join(path, img_name))
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

# Load your dataset
dataset_path = r"C:\SIH Project\Dataset\archive (2)\CCTV Gender Classifier Dataset"
X, y = load_data(dataset_path)

# Normalize pixel values
X = X.astype('float32') / 255.0

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to categorical
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [5]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

In [6]:
# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

C:\Users\ybadr\anaconda3\Anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

In [8]:
# Train the model
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=EPOCHS,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/100


C:\Users\ybadr\anaconda3\Anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


479/479 ━━━━━━━━━━━━━━━━━━━━ 110s 218ms/step - accuracy: 0.5800 - loss: 1.1234 - val_accuracy: 0.5665 - val_loss: 0.8543 - learning_rate: 1.0000e-04
Epoch 2/100
479/479 ━━━━━━━━━━━━━━━━━━━━ 106s 220ms/step - accuracy: 0.6192 - loss: 0.8746 - val_accuracy: 0.6643 - val_loss: 0.6335 - learning_rate: 1.0000e-04
Epoch 3/100
479/479 ━━━━━━━━━━━━━━━━━━━━ 106s 220ms/step - accuracy: 0.6165 - loss: 0.8164 - val_accuracy: 0.6771 - val_loss: 0.6219 - learning_rate: 1.0000e-04
Epoch 4/100
479/479 ━━━━━━━━━━━━━━━━━━━━ 104s 215ms/step - accuracy: 0.6357 - loss: 0.7474 - val_accuracy: 0.6957 - val_loss: 0.5645 - learning_rate: 1.0000e-04
Epoch 5/100
479/479 ━━━━━━━━━━━━━━━━━━━━ 106s 220ms/step - accuracy: 0.6511 - loss: 0.7076 - val_accuracy: 0.6855 - val_loss: 0.5995 - learning_rate: 1.0000e-04
Epoch 6/100
479/479 ━━━━━━━━━━━━━━━━━━━━ 106s 219ms/step - accuracy: 0.6528 - loss: 0.6787 - val_accuracy: 0.7153 - val_loss: 0.5553 - learning_rate: 1.0000e-04
Epoch 7/100
479/479 ━━━━━━━━━━━━━━━━━━━━ 105s 

In [10]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")

# Save the model
model.save('cctv_gender_classifier.h5')

120/120 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.8870 - loss: 0.2656


Test accuracy: 0.8808
